In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'F:\Fingertips\python\train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = df.dropna()

In [5]:
text = df.copy()

In [6]:
text.reset_index(inplace=True)

In [7]:
# Independent feature
x = text.drop('label', axis=1)
x.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [8]:
# Dependent feature
y = df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.6.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
# Vocabulary size
voc_size = 5000

In [13]:
# Data preprocessing
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


In [14]:
ps = PorterStemmer()
corpus = []

In [15]:
for i in range(len(text)):
        review = re.sub('[^a-zA-z]', ' ', text['title'][i])
        review = review.lower()
        review = review.split()
        review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
        

In [17]:
onehot_rp = [one_hot(words,voc_size) for words in corpus]
onehot_rp

[[1509, 624, 1282, 2154, 2876, 444, 2429, 1561, 4417, 3338],
 [4365, 3293, 34, 4290, 1358, 871, 518],
 [4222, 802, 3062, 336],
 [4359, 4267, 4242, 2481, 420, 4855],
 [4695, 1358, 2984, 4152, 3156, 2360, 1358, 1022, 3461, 2161],
 [1801,
  2743,
  348,
  2734,
  4872,
  3328,
  2118,
  297,
  2134,
  4784,
  4224,
  3028,
  1863,
  4645,
  518],
 [4751, 2327, 4143, 2654, 3538, 1868, 2616, 4353, 1483, 3334, 2439],
 [811, 2750, 612, 1359, 4717, 3675, 3328, 3667, 1483, 3334, 2439],
 [1632, 3020, 2470, 2462, 2230, 4433, 3213, 3155, 3328, 1428],
 [5, 849, 2732, 242, 3527, 3756, 2692, 2741],
 [3305, 2681, 4222, 3230, 1184, 2940, 1643, 2189, 898, 3125, 2078],
 [2481, 3479, 2876, 4433, 3328, 4717],
 [1349, 2790, 4139, 124, 4682, 3205, 3051, 2058, 2053],
 [3656, 2422, 2732, 273, 1325, 3302, 624, 1483, 3334, 2439],
 [2853, 3804, 1916, 1409, 4042, 1483, 3334, 2439],
 [2835, 3764, 4987, 3222, 2503, 735, 314, 2684, 2154, 4467],
 [1874, 986, 3293],
 [1629, 1540, 2302, 2934, 3328, 2070, 3468, 518],
 [3

# Embendding Representation

In [19]:
sent_len = 20
embedded_docs = pad_sequences(onehot_rp, padding='pre', maxlen=sent_len)

In [21]:
embedded_docs

array([[   0,    0,    0, ..., 1561, 4417, 3338],
       [   0,    0,    0, ..., 1358,  871,  518],
       [   0,    0,    0, ...,  802, 3062,  336],
       ...,
       [   0,    0,    0, ..., 1483, 3334, 2439],
       [   0,    0,    0, ..., 3381, 3622, 1807],
       [   0,    0,    0, ..., 3129, 3140, 2233]])

In [22]:
# Creating model
embedding_vector_feature = 50
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_feature, input_length = sent_len))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 50)            250000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________


In [25]:
import numpy as np

In [26]:
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=50)

In [28]:
# final training
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=100)

Epoch 1/10
123/123 [==============================] - 8s 31ms/step - loss: 0.3683 - accuracy: 0.8263 - val_loss: 0.1997 - val_accuracy: 0.9133
Epoch 2/10
123/123 [==============================] - 3s 24ms/step - loss: 0.1428 - accuracy: 0.9429 - val_loss: 0.1972 - val_accuracy: 0.9147
Epoch 3/10
123/123 [==============================] - 3s 27ms/step - loss: 0.0991 - accuracy: 0.9633 - val_loss: 0.2333 - val_accuracy: 0.9176
Epoch 4/10
123/123 [==============================] - 3s 24ms/step - loss: 0.0715 - accuracy: 0.9731 - val_loss: 0.2639 - val_accuracy: 0.9163
Epoch 5/10
123/123 [==============================] - 3s 26ms/step - loss: 0.0536 - accuracy: 0.9816 - val_loss: 0.2690 - val_accuracy: 0.9118
Epoch 6/10
123/123 [==============================] - 3s 26ms/step - loss: 0.0338 - accuracy: 0.9890 - val_loss: 0.3426 - val_accuracy: 0.9127
Epoch 7/10
123/123 [==============================] - 3s 27ms/step - loss: 0.0250 - accuracy: 0.9915 - val_loss: 0.3006 - val_accuracy: 0.9115

In [45]:
# Performance metrics and accuracy
y_pred = model.predict(x_test)
y_pred

array([[9.9995911e-01],
       [2.8135162e-08],
       [1.7428860e-05],
       ...,
       [1.3610721e-04],
       [7.5431412e-09],
       [6.1450116e-09]], dtype=float32)

In [46]:
threshold =0.5 # this can be changed. For simplistic example, let uss consider 0.5
y_pred = np.where(y_pred<threshold,0,1)
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [50]:
from sklearn.metrics import confusion_matrix

In [51]:
confusion_matrix(y_test,y_pred)

array([[3217,  291],
       [ 263, 2264]], dtype=int64)

In [48]:
from sklearn.metrics import accuracy_score

In [49]:
accuracy_score(y_test,y_pred)

0.9082021541010771